In [2]:
from IPython.display import display, HTML, Image
import base64
from google.colab import files
import os

# Function to display HTML
def display_html(html):
    display(HTML(html))

# HTML code for a webpage with image folder upload and label entry
html_code = """
<!DOCTYPE html>
<html>
<head>
    <title>Object Detection Web App</title>
</head>
<body>
    <h1>Welcome to Object Detection Web App</h1>
    <p>This is a simple web page created in Google Colab.</p>

    <form action="javascript:void(0);" id="upload-form" enctype="multipart/form-data">
        <input type="file" id="folder-input" webkitdirectory directory multiple>
        <input type="text" id="label-input" placeholder="Enter Label">
        <input type="button" value="Upload Images" onclick="uploadImages()">
    </form>

    <h2>Uploaded Images</h2>
    <div id="image-gallery"></div>

    <script>
        function uploadImages() {
            var folderInput = document.getElementById('folder-input');
            var labelInput = document.getElementById('label-input');
            var gallery = document.getElementById('image-gallery');

            var files = folderInput.files;
            var label = labelInput.value.trim();

            if (label === "") {
                alert("Please enter a label for the images.");
                return;
            }

            for (var i = 0; i < files.length; i++) {
                var reader = new FileReader();

                reader.onload = function(){
                    var img = document.createElement('img');
                    img.src = reader.result;
                    gallery.appendChild(img);
                };

                reader.readAsDataURL(files[i]);
            }

            // Save images to the Colab environment with labels as folder names
            var formData = new FormData();
            formData.append('label', label);
            for (var i = 0; i < files.length; i++) {
                formData.append('images', files[i]);
            }

            fetch('/upload', {
                method: 'POST',
                body: formData
            }).then(response => response.text())
            .then(data => console.log(data))
            .catch(error => console.error('Error:', error));
        }
    </script>
</body>
</html>
"""

# Save HTML code to an HTML file
html_file_path = '/content/object_detection_web_app.html'
with open(html_file_path, 'w') as f:
    f.write(html_code)

# Display a link to download the HTML file
display_html(f'<a href="{html_file_path}" target="_blank">Download HTML File and Open Locally</a>')

# Image upload logic
uploaded = files.upload()
for label, images in uploaded.items():
    label_dir = os.path.join('/content/', label)
    os.makedirs(label_dir, exist_ok=True)
    for image_name, image_data in images.items():
        image_path = os.path.join(label_dir, image_name)
        with open(image_path, 'wb') as f:
            f.write(image_data)


ModuleNotFoundError: No module named 'google'

In [2]:
pip install edgeimpulse


     ---------------------------------------- 0.0/44.7 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.7 kB ? eta -:--:--
     ----------------- -------------------- 20.5/44.7 kB 217.9 kB/s eta 0:00:01
     -------------------------------------- 44.7/44.7 kB 313.3 kB/s eta 0:00:00
     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.2 MB 991.0 kB/s eta 0:00:02
     -- ------------------------------------- 0.1/1.2 MB 812.7 kB/s eta 0:00:02
     ---- ----------------------------------- 0.1/1.2 MB 1.1 MB/s eta 0:00:02
     ----- ---------------------------------- 0.2/1.2 MB 1.2 MB/s eta 0:00:01
     --------- ------------------------------ 0.3/1.2 MB 1.3 MB/s eta 0:00:01
     ----------- ---------------------------- 0.4/1.2 MB 1.3 MB/s eta 0:00:01
     ----------------- ---------------------- 0.5/1.2 MB 1.7 MB/s eta 0:00:01
     ----------------------- ---------------- 0.7/1.2 MB 1.9 MB/s eta 0:00


[notice] A new release of pip is available: 23.0 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install git

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement git (from versions: none)
ERROR: No matching distribution found for git

[notice] A new release of pip is available: 23.0 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
pip install tensorflow.keras


Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement tensorflow.keras (from versions: none)
ERROR: No matching distribution found for tensorflow.keras
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:

from tensorflow.keras import layers, models, optimizers
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
                   
# Function to load images and labels from folders
def load_data(folder_path):
    images = []
    labels = []
    
    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)
        if os.path.isdir(label_path):
            for filename in os.listdir(label_path):
                img_path = os.path.join(label_path, filename)
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
                images.append(img)
                labels.append(label)
                
    return np.array(images), np.array(labels)


# Load data from folders
data_path = "/Users/varunsankar/webappml/data"
images, labels = load_data(data_path)

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)

# Normalize pixel values to be between 0 and 1
X_train, X_test = X_train / 255.0, X_test / 255.0

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the ImageDataGenerator on the training data
datagen.fit(X_train)

                   
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(len(set(labels)), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with augmented data
model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_data=(X_test, y_test))

# Save the model
model.save("01objcls.h5")

print("Model trained with data augmentation and saved successfully.")
    

Epoch 1/10
1/1 [==============================] - 0s 398ms/step - loss: 0.6844 - accuracy: 0.6667 - val_loss: 0.7236 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 24ms/step - loss: 0.6936 - accuracy: 0.5000 - val_loss: 0.7161 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 24ms/step - loss: 0.6760 - accuracy: 0.6667 - val_loss: 0.7166 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 22ms/step - loss: 0.6790 - accuracy: 0.6667 - val_loss: 0.7060 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 22ms/step - loss: 0.6759 - accuracy: 0.8333 - val_loss: 0.6996 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 23ms/step - loss: 0.6701 - accuracy: 0.6667 - val_loss: 0.6999 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 22ms/step - loss: 0.6721 - accuracy: 0.6667 - val_loss: 0.6896 - val_accuracy: 0.5000
Epoch 8/10
1/1 [==========

/Users/varunsankar/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
from flask import Flask, render_template, request, jsonify
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
import os
from werkzeug.utils import secure_filename
import subprocess
from flask import send_file
from tensorflow import keras
import edgeimpulse as ei
import tensorflow as tf

In [9]:
loaded_model = tf.keras.models.load_model(hs1.h5)

model_output_type = ei.model.output_type.Classification(labels=labels_list)

    # Set model input type
model_input_type = ei.model.input_type.OtherInput()

    # Estimate the RAM, ROM, and inference time for our model on the target hardware family
try:
        profile = ei.model.profile(model=loaded_model, device=deploy_target)
        print(profile.summary())
except Exception as e:
        print(f"Could not profile: {e}")

    # Create C++ library with trained model
deploy_bytes = None
try:
        deploy_bytes = ei.model.deploy(model=loaded_model, model_output_type=model_output_type,
                                       model_input_type=model_input_type, deploy_target="arduino")
except Exception as e:
        print(f"Could not deploy: {e}")

    # Write the downloaded raw bytes to a temporary file
if deploy_bytes:
        temp_deploy_filename = deploy_filename
        with open(temp_deploy_filename, 'wb') as f:
            f.write(deploy_bytes.getvalue())



NameError: name 'hs1' is not defined